In [581]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
import json
import matplotlib.pyplot as plt
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator


# color pallette
cnf, dth, rec, act = '#393e46', '#ff2e63', '#21bf73', '#fe9801' 
DEFAULT_PLOTLY_COLORS=['rgb(31, 119, 180)', 'rgb(255, 127, 14)',
                       'rgb(44, 160, 44)', 'rgb(214, 39, 40)',
                       'rgb(148, 103, 189)', 'rgb(140, 86, 75)',
                       'rgb(227, 119, 194)', 'rgb(127, 127, 127)',
                       'rgb(188, 189, 34)', 'rgb(23, 190, 207)']

# 기본 폰트 설정
layout_font = {'font':dict(size=12,color='#60606e',family='Malgun Gothic' )}


In [549]:
df18 = pd.read_csv('서울시 우리마을가게 상권분석서비스(상권-추정매출)_2018.csv')
df19 = pd.read_csv('서울시 우리마을가게 상권분석서비스(상권-추정매출)_2019.csv')
df20 = pd.read_csv('서울시우리마을가게상권분석서비스(상권-추정매출)_2020.csv')

df1 = pd.read_csv('행정동코드.csv')
df2= pd.read_csv('상권영역.csv',index_col=False)

In [550]:
code = df1[df1["CT_NM(시군구명)"]=='용산구']
code.rename(columns={'H_SDNG_CD(통계청행정동코드)':"통계청행정동코드"},inplace=True)
code.rename(columns={'H_DNG_CD(행자부행정동코드)':"행정동코드"},inplace=True)
code.rename(columns={"DO_NM(시도명)":"시도명"},inplace=True)
code.rename(columns={"CT_NM(시군구명)":"시군구명"},inplace=True)
code.rename(columns={"H_DNG_NM(행정동명)":"행정동명"},inplace=True)

df2 = pd.merge(df2,code.iloc[:,1:], how='left', left_on='ADSTRD_CD', right_on='행정동코드')
df2 = df2.dropna()

In [551]:
df18 = pd.merge(df18,df2, how='left', left_on='상권_코드', right_on='TRDAR_CD')
df18 = df18.dropna()
df19 = pd.merge(df19,df2, how='left', left_on='상권_코드', right_on='TRDAR_CD')
df19 = df19.dropna()
df20 = pd.merge(df20,df2, how='left', left_on='상권_코드', right_on='TRDAR_CD')
df20 = df20.dropna()

In [552]:
df_18 = df18.query('(행정동명 =="이태원1동")|(행정동명 =="이태원2동")')
df_19 = df19.query('(행정동명 =="이태원1동")|(행정동명 =="이태원2동")')
df_20 = df20.query('(행정동명 =="이태원1동")|(행정동명 =="이태원2동")')

In [554]:
df_18 = pd.DataFrame(df_18.groupby(['기준_분기_코드','상권_구분_코드_명'], as_index=False)['분기당_매출_금액'].mean())
df_19 = pd.DataFrame(df_19.groupby(['기준_분기_코드','상권_구분_코드_명'], as_index=False)['분기당_매출_금액'].mean())
df_20 = pd.DataFrame(df_20.groupby(['기준_분기_코드','상권_구분_코드_명'], as_index=False)['분기당_매출_금액'].mean())
# df.groupby(['시간대구분','행정동명'])['총생활인구수'].transform('mean')


In [555]:
df_18['분기당_매출_금액'] = df_18['분기당_매출_금액'].astype('int64') 
df_19['분기당_매출_금액'] = df_19['분기당_매출_금액'].astype('int64') 
df_20['분기당_매출_금액'] = df_20['분기당_매출_금액'].astype('int64') 

In [556]:
df_19['전년도 분기 대비 매출증감액'] = df_19["분기당_매출_금액"].values - df_18["분기당_매출_금액"].values

In [557]:
df_19['전년도 분기 대비 매출증감율'] = (df_19['전년도 분기 대비 매출증감액'].values / df_18["분기당_매출_금액"].values) * 100

In [558]:
df_20['전년도 분기 대비 매출증감액'] = df_20["분기당_매출_금액"].values - df_19["분기당_매출_금액"].values
df_20['전년도 분기 대비 매출증감율'] = (df_20['전년도 분기 대비 매출증감액'].values / df_19["분기당_매출_금액"].values) * 100

In [559]:
def year_add(x):
    return '2019년'+str(x)+'분기'

df_19['기준_분기_코드'] = df_19['기준_분기_코드'].apply(year_add)

In [560]:
def year_add(x):
    return '2020년'+str(x)+'분기'

df_20['기준_분기_코드'] = df_20['기준_분기_코드'].apply(year_add)

In [561]:
result = pd.concat([df_19,df_20])

In [574]:
result['전년도 분기 대비 매출증감율'] = result['전년도 분기 대비 매출증감율'].round(1)

In [576]:
result1= result.query('상권_구분_코드_명=="관광특구"')
result2= result.query('상권_구분_코드_명=="발달상권"')
result3= result.query('상권_구분_코드_명=="골목상권"')
result4= result.query('상권_구분_코드_명=="전통시장"')
#,labels={'기준_분기_코드':'분기별','분기당_매출_금액':'분기당평균매출금액(천원)',


In [582]:

# fig = go.Figure()
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=("관광특구", "발달상권", "골목상권", "전통시장"),
                    specs=[[{"secondary_y": True}, {"secondary_y": True}],
                           [{"secondary_y": True}, {"secondary_y": True}]])

## 관광특구
fig.add_trace(
    go.Bar(x=result1['기준_분기_코드'],
            y=result1['전년도 분기 대비 매출증감액'],
            name='관광특구 증감액'),
            row=1, col=1,
            secondary_y=False)
fig.add_trace(
    go.Scatter(x=result1['기준_분기_코드'],
            y=result1['전년도 분기 대비 매출증감율'],
            mode='lines+markers+text',
            name='관광특구 증감율',
            text=result1['전년도 분기 대비 매출증감율'],
            textposition="bottom center"),
            row=1, col=1,
            secondary_y=True)
## 발달상권
fig.add_trace(
    go.Bar(x=result2['기준_분기_코드'],
            y=result2['전년도 분기 대비 매출증감액'],
            name='발달상권 증감액'),
            row=1, col=2,
            secondary_y=False)
fig.add_trace(
    go.Scatter(x=result2['기준_분기_코드'],
            y=result2['전년도 분기 대비 매출증감율'],
            mode='lines+markers+text',
            name='발달상권 증감율',
            text=result2['전년도 분기 대비 매출증감율'],
            textposition="bottom center"),
            row=1, col=2,
            secondary_y=True)
## 골목상권    
fig.add_trace(
    go.Bar(x=result3['기준_분기_코드'],
            y=result3['전년도 분기 대비 매출증감액'],
            name='골목상권 증감액'),
            row=2, col=1,
            secondary_y=False)
fig.add_trace(
    go.Scatter(x=result3['기준_분기_코드'],
            y=result3['전년도 분기 대비 매출증감율'],
            mode='lines+markers+text',
            name='골목상권 증감율',
            text=result3['전년도 분기 대비 매출증감율'],
            textposition="bottom center"),
            row=2, col=1,
            secondary_y=True)

## 전통시장
fig.add_trace(
    go.Bar(x=result4['기준_분기_코드'],
            y=result4['전년도 분기 대비 매출증감액'],
            name='전통시장 증감액'),
            row=2, col=2,
            secondary_y=False)
fig.add_trace(
    go.Scatter(x=result4['기준_분기_코드'],
            y=result4['전년도 분기 대비 매출증감율'],
            mode='lines+markers+text',
            name='전통시장 증감율',
            text=result4['전년도 분기 대비 매출증감율'],
            textposition="bottom center"),
            row=2, col=2,
            secondary_y=True)


fig.update_layout(title='<b>2019~2020 이태원 분기별 매출 증감액과 증감율 </b>', height=700,**layout_font)

# Set y-axes titles
fig.update_yaxes(title_text="증감액(천원)", secondary_y=False)
fig.update_yaxes(title_text="증감율(%)", secondary_y=True)
fig.show()